In [1]:
import pandas as pd
import os
import subprocess

In [16]:
filename  = '/global/homes/b/bpb/repos/envnet/scripts/my_files_test.csv'
df = pd.read_csv(filename)
for c in df.columns:
    print('Column Heading:',c)
    print('  ',df.loc[0,c])





Column Heading: parquet
   /global/cfs/cdirs/metatlas/projects/carbon_network/raw_data/metatlas/20230223_EB_MdR_101544-059_SynDAC_20230223_QE144_C18-EP_USDAY72350_NEG_MS2_29_RS-HA-NA_1__18_deconvoluted.parquet
Column Heading: h5
   /global/cfs/cdirs/metatlas/projects/carbon_network/raw_data/metatlas/20230223_EB_MdR_101544-059_SynDAC_20230223_QE144_C18-EP_USDAY72350_NEG_MS2_29_RS-HA-NA_1__18.h5
Column Heading: environmental_subclass
   Suwannee River Humic Acid Standard III
Column Heading: original_file_type
   h5
Column Heading: lcmsrun_observed
   metatlas/20230223_EB_MdR_101544-059_SynDAC_20230223_QE144_C18-EP_USDAY72350_NEG_MS2_29_RS-HA-NA_1__18


In [22]:

filename = '/global/homes/b/bpb/repos/envnet/scripts/my_files.csv'
df = pd.read_csv(filename)
# replace all text having 'carbon_network/raw_data' with 'envnet_build_files' in the entire dataframe
for c in df.columns:
    df[c] = df[c].str.replace('carbon_network/raw_data', 'envnet_build_files', regex=False)
df = df[~df['parquet'].str.contains('MSV000081030')]
idx = df['parquet'].str.contains('_deconvoluted.parquet')
df.loc[~idx,'parquet'] = df.loc[~idx,'parquet'].str.replace('.parquet','_deconvoluted.parquet', regex=False)
df['lcmsrun_observed'] = df['h5'].apply(lambda x: x.replace('.h5',''))
df['lcmsrun_observed'] = df['lcmsrun_observed'].str.replace('/global/cfs/cdirs/metatlas/projects/envnet_build_files/','', regex=False)
df['original_file_type'] = 'h5'
output_path = '/global/homes/b/bpb/repos/envnet/results/full_build_20250908_181404'
output_filename = os.path.join(output_path,'input_for_annotation.csv')
df.to_csv(output_filename, index=False)
print(f'Wrote {output_filename} with {len(df)} rows')

Wrote /global/homes/b/bpb/repos/envnet/results/full_build_20250908_181404/input_for_annotation.csv with 1389 rows


In [ ]:


# use recursive find to get all parquet files
# use linux find since it is much faster than glob
cmd = "find /global/cfs/cdirs/metatlas/projects/envnet_build_files/ -name '*_deconvoluted.parquet'"
# run command
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
files = result.stdout.splitlines()
len(files)
files = [f for f in files if "MSV000081030" not in f] # MetaboLights MTBLS144 - GNPS Dissolved organic matter produced by Thalassiosira pseudonana


# filename = '/global/homes/b/bpb/repos/envnet/scripts/my_files.csv'
# df = pd.read_csv(filename)
# # replace all text having 'carbon_network/raw_data' with 'envnet_build_files' in the entire dataframe
# for c in df.columns:
#     df[c] = df[c].str.replace('carbon_network/raw_data', 'envnet_build_files', regex=False)
# df = df[~df['parquet'].str.contains('MSV000081030')]
# idx = df['parquet'].str.contains('_deconvoluted.parquet')
# df.loc[~idx,'parquet'] = df.loc[~idx,'parquet'].str.replace('.parquet','_deconvoluted.parquet', regex=False)

# print(df.shape)
# df.to_csv('/global/homes/b/bpb/repos/envnet/scripts/build_files.csv', index=False)
# for f in df.head()['parquet'].tolist():
#     print(f)
#     print(os.path.exists(f))

(1389, 3)
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230223_EB_MdR_101544-059_SynDAC_20230223_QE144_C18-EP_USDAY72350_NEG_MS2_29_RS-HA-NA_1__18_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20221219_EB_MdR_101544-059_PyrToDAC_20221219_EXP120A_C18-EP_USDAY63672_NEG_MS2_8_FAIII-1mg-NA_1__48_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230113_EB_MdR_101544-059_FRCgwater_20230105_EXP120A_C18-EP_USDAY72349_NEG_MS2_13_SRFA3-H2O-fil-NA-NA_1__18_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230113_EB_MdR_101544-059_FRCgwater_20230105_EXP120A_C18-EP_USDAY72349_NEG_MS2_14_SRFA3-H2O-unfil-NA-NA_1__36_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230113_EB_MdR_101544-059_FRCgwater_20230105_EXP120A_C18-EP_USDAY72349_NEG_MS2_15_SRFA3-MeOH-fil-NA-NA_1__12_deconvoluted.parquet
True


In [ ]:
# # remove MSV000081030 files.  These seem to be not environmental

print(len(files))
# files = [f for f in files if "MSV000088008" not in f]

# # files = [f for f in files if "DOM_Interlab-LCMS_Lab024" not in f]
# len(files)

In [ ]:
import os
for f in files:
    mzml_filename = f.replace("_deconvoluted.parquet", ".mzML")
    h5_filename = f.replace("_deconvoluted.parquet", ".h5")
    if not os.path.exists(mzml_filename):
        print(f)
    if not os.path.exists(h5_filename):
        print(f)